In [46]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import spacy
import spacy_spanish_lemmatizer
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import nltk
nltk.download('punkt')

In [ ]:
accent_mapping = {
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'ü': 'u',  # Remove umlaut from 'ü'
        'Á': 'A',
        'É': 'E',
        'Í': 'I',
        'Ó': 'O',
        'Ú': 'U',
        'Ü': 'U'   # Remove umlaut from 'Ü'
    }

In [47]:
df = pd.read_csv('sentiment_integrado.csv')
df.head(10)

,Unnamed: 0,id,texto,sentimiento,Cantidad palabras,cant caracteres especiales,cant menciones,cant hashtags
0,0,142389495503925248,"Salgo de #VeoTV , que día más largoooooo...",Indefinido,8,4,0,1
1,1,142389933619945473,@PauladeLasHeras No te libraras de ayudar me/n...,Neutral,10,2,1,0
2,2,142391947707940864,@marodriguezb Gracias MAR,Positivo,3,0,1,0
3,3,142416095012339712,"Off pensando en el regalito Sinde, la que se v...",Negativo,24,4,0,0
4,4,142422495721562112,Conozco a alguien q es adicto al drama! Ja ja ...,Positivo,15,2,0,0
5,5,142424715175280640,"RT @FabHddzC: Si amas a alguien, déjalo libre....",Indefinido,16,6,2,0
6,6,142483342040907776,Toca @crackoviadeTV3 . Grabación dl especial N...,Positivo,8,5,1,0
7,7,142493511634259968,Hoy asisitiré en Madrid a un seminario sobre l...,Indefinido,16,1,0,0
8,8,142494476051562496,Buen día todos! Lo primero mandar un abrazo gr...,Positivo,25,2,1,0
9,9,142496796416016384,Desde el escaño. Todo listo para empezar #endi...,Positivo,12,6,0,1


In [50]:
mappeo_sent = { "Neutral": 0, "Negativo": 1, "Positivo": 2}

In [48]:
df["sentimiento"] = df["sentimiento"].apply(lambda x: x if x != "Indefinido" else "Neutral")
df.drop("Unnamed: 0", axis = 1, inplace = True)
df["sentimiento"] = df["sentimiento"].apply(lambda x: mappeo_sent[x])

In [51]:
df

sentimiento
1    5798
2    5680
0    5592
Name: count, dtype: int64

In [54]:
stop_words = set(stopwords.words('spanish'))
stop_words

In [58]:
nlp = spacy.load("es_core_news_sm")
nlp.replace_pipe("lemmatizer", "spanish_lemmatizer")

In [62]:
def remove_mentions(sentence):
    # Define regular expression pattern to match mentions
    mention_pattern = r'@\w+'
    
    # Substitute mentions with an empty string
    cleaned_sentence = re.sub(mention_pattern, '', sentence)
    
    return cleaned_sentence


In [70]:
train, test_valid_data = train_test_split(df[['texto', 'sentimiento']].dropna(), test_size=0.4, random_state=42)
test, validation = train_test_split(test_valid_data, test_size=0.5, random_state=42)

In [74]:
def remove_mentions(sentence):
    # Define regular expression pattern to match mentions
    mention_pattern = r'@\w+'
    
    # Substitute mentions with an empty string
    cleaned_sentence = re.sub(mention_pattern, '', sentence)
    
    return cleaned_sentence


In [113]:
def remove_stopwords(tokens):
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token != ' ']
    
    return filtered_tokens

In [99]:
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation))

In [86]:
def lemmatize(texto):
    return [token.lemma_ for token in texto]

In [114]:
def normalizar_palabra(sentence):
    sentence = re.sub(r'\$\w*', '', sentence)
    # remove old style retweet text "RT"
    sentence = re.sub(r'^RT[\s]+', '', sentence)
    # remove hyperlinks
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
    # remove hashtags
    # only removing the hash # sign from the word
    sentence = re.sub(r'#', '', sentence)
    sentence = re.sub(r'\d+', '', sentence)
    sentence = sentence.lower()

    for accented_char, unaccented_char in accent_mapping.items():
        sentence = sentence.replace(accented_char, unaccented_char)

    sentence = remove_mentions(sentence)
    sentence = remove_punctuation(sentence)
    tokens = nlp(sentence)
    tokens = lemmatize(tokens)
    tokens = remove_stopwords(tokens)

    sentence = ' '.join(tokens)

    print('texto: ', sentence)
    
    return sentence

In [91]:
train

,texto,sentimiento
10221,"@Manuellflorod Bienvenida (triste) realidad, a...",1
9565,Estar en los brazos de mi novio es lo único qu...,2
7555,"Todos tenemos un pasado @ahorapodemos ,que se ...",0
7713,@ahorapodemos @ierrejon Tambien que las empres...,2
2825,En Islandia: juicio a los responsables de la ...,0
...,...,...
11285,"No, solo quiero bailarte la medusa loca en pri...",2
11965,@cuervotinelli lo único que te puse fue que qu...,1
5390,La Audiencia Nacional condena a 20 años de pri...,1
860,RT @eP_Titulares: #Sociedad Muere apuñalada la...,1


In [115]:
train['texto_normalizado'] = train.texto.apply(lambda x: normalizar_palabra(x))

texto:  bienvenida triste realidad andar mismo
texto:  brazo novio unico querer necesitar
texto:  tener pasado decir
texto:     tambien empresa independiente contactar si estubieran interesado seguro avanzariamos
texto:  islandia juicio responsable crisis negativa pago deuda elaboracion nueva constitucion camino
texto:  tono duro concesión replica rajoy amaiur anunciar encarara relacion sera logico peor
texto:  comenzar intervencion acto cierre campaña    siguelo directo
texto:  bdias em ira puente si ir dejeis llevar tableta pc luego orbyt momento decir mas
texto:  ff demostrar penultimo tweet prejuicio oportunidad actualizacion
texto:  herencia    trabajo rato euro hora editorial    h ¡ gracias zapatero
texto:  tumbes traer bueno recuerdo
texto:  feliz dia inocente algún habeis picado pelo seguir igual jaaaaaaa
texto:  regalo navidad solo querer flotador unicornio
texto:  rajoy admitir pensar sindicato pueden convocar huelga reforma laboral serio irresponsable si hacer
texto:  buen d

In [116]:
validation['texto_normalizado'] = validation.texto.apply(lambda x: normalizar_palabra(x))

texto:  satisfecho
texto:  mejor pasar navidad familia amigo uds pasar
texto:  planeta creativo nuevo placer interesante jornada mañana pronto seguir punto
texto:  necesitar ir hogar dormir dos hijo peludos kabek gripe
texto:  xii edicion premio culturas extremadura galardon chojin
texto:  tamales pierna fresquitos queque
texto:  peor mejor amigo tener whats tener hablar stalker consejo sarcasticos
texto:  esperar previsible referir previsible resolución ajustar querer politico
texto:  saludito perù bello abrazo querer rt    saludo peru maestro
texto:  encantar arregla julio reyes hacer buen equipo gran acierto trabajar nuevo disco
texto:  bueno dias si permites corazon brujula mente mapa alma guia entonces nunca perderas besitos
texto:     haber enterar quereis cambiar discutible tambien
texto:  publicar foto nueva facebook
texto:  promoción sesión romanticas enero febrero daniamaciel generandorecuerdos
texto:  bueno dias princesa inevitable tenia decir frase lavidaesbella
texto:  rt


In [117]:
test['texto_normalizado'] = test.texto.apply(lambda x: normalizar_palabra(x))

texto:  funcionario ademas tener congelado sueldo tendran trabajar dos horas media mas practica recorte salarial
texto:  solidaridad familia d marta castillohoy sido dia duro mayoria ciudadanosimpotentes sentencia
texto:  arrancar segundo termo vengo finde largo dar calentarse buen dia
texto:  bonos emitido centralizadamente x tesoro europeo respaldo conjunto d paises rt
texto:  va explicar programa voy votar nunca prestige
texto:  mes español sacar millón euro españa
texto:  bigote va declarar solo decir nunca regalar traje camps costa campo betoret juiciocamps sentar anterior
texto:  esperar momento primer capitulo dije mateo indicado anita
texto:  recargarpsoe verdad post buzon voz
texto:  juas ahora decir paja van hacer tracer novia siempre puede caer mas bajo aun ¿
texto:  mayorias absoluto venir ser mismo dictadura
texto:     ¡ pais niña rajoy m prima español disparar primo rajoy ir españa
texto:  sentimiento orgullo valencia grande querer mejor valencia
texto:  ver persona salud

In [ ]:
train.to_csv('train.csv')

In [ ]:
validation.to_csv('validation.csv')

In [ ]:
test.to_csv('test.csv')